In [3]:
# !pip install pystan

In [8]:
# import pystan                                                                                                                                        
# import pandas as pd                                                                                  
# import numpy as np                                                                                   
                                                                                                     
# code = """                                                                                         
# data {                                                                                               
#   int N; //the number of training observations                                                       
#   int N2; //the number of test observations                                                          
#   int K; //the number of features                                                                    
#   int y[N]; //the response                                                                           
#   matrix[N,K] X; //the model matrix                                                                  
#   matrix[N2,K] new_X; //the matrix for the predicted values                                          
# }                                                                                                    
# parameters {                                                                                         
#   real alpha;                                                                                        
#   vector[K] beta; //the regression parameters                                                        
# }                                                                                                    
# transformed parameters {                                                                             
#   vector[N] linpred;                                                                                 
#   linpred = alpha+X*beta;                                                                            
# }                                                                                                    
# model {                                                                                              
#   alpha ~ cauchy(0,10); //prior for the intercept following Gelman 2008                              
                                                                                                     
#   for(i in 1:K)                                                                                      
#     beta[i] ~ student_t(1, 0, 0.03);                                                                 
                                                                                                     
#   y ~ bernoulli_logit(linpred);                                                                      
# }                                                                                                    
# generated quantities {                                                                               
#   vector[N2] y_pred;                                                                                 
#   y_pred = alpha+new_X*beta; //the y values predicted by the model                                   
# }                                                                                                    
# """          

# train = pd.read_csv('train.csv', )                                                            
# train.pop('id')                                                                                      
# target = train.pop('target').astype(int)                                                             
                                                                                                     
# test = pd.read_csv('test.csv')                                                              
# ids = test.pop('id')                                                                                 
                                                                                                     
# data = {                                                                                             
#     'N': 250,                                                                                        
#     'N2': 19750,                                                                                     
#     'K': 300,                                                                                        
#     'y': target,                                                                                     
#     'X': train,                                                                                      
#     'new_X': test,                                                                                   
# }                                                                                                    
                                                                                                     
# sm = pystan.StanModel(model_code=code)                                                               
# fit = sm.sampling(data=data, seed=1234)                                                              
# ex = fit.extract(permuted=True)                                                                      
# target = np.mean(ex['y_pred'], axis=0)                                                               
# df = pd.DataFrame({'id': ids, 'target': target})                                                     
# # df[['id', 'target']].to_csv('submission.csv', index=False)

In [ ]:
import pystan
import pandas as pd
import numpy as np
import input_data # custom module with data pre-processing functions

from sklearn.metrics import roc_auc_score
from scipy.special import expit

log_ab_normal = 
    """
    /* Stan model code for logistic regression */
    data {                                 
        int<lower=0> N;  // count of observations
        int<lower=0> K;  // count of features
        matrix[N, K] X;  // feature matrix
        int<lower=0,upper=1> y[N];  // target
    }
    parameters {
        real alpha;  // bias
        vector[K] beta;  // feature weights
    }
    model { 
        alpha ~ normal(0,5);  // bias prior
        beta ~ normal(0,1.0);  // featue weights prior
        y ~ bernoulli_logit(alpha + X * beta);  // likelihood
    }
    generated quantities {}
    """


def fit(input_fn, hparams):
    """Fit a Bayesian logistic regression model using MCMC sampling.
   	Args:
		input_fn: lambda function that return training data as two numpy arrays (x, y).
		
	Returns:
		fit: Fitted Stan output.	
	"""

    features, labels = input_fn()

    stan_datadict = {}
    stan_datadict['N'] = features.shape[0]
    stan_datadict['K'] = features.shape[1]
    stan_datadict['X'] = features.values
    stan_datadict['y'] = labels.values

    model = pystan.StanModel(model_code=log_ab_normal)

    fit = model.sampling(
        data=stan_datadict, warmup=250, iter=1000, verbose=True)

    return fit


def evaluate(fit, input_fn):
    """Evaluate the performance of fitted model on unseen test data.
   	Args:
		input_fn: lambda function that outputs test data as two numpy arrays (x, y).
		
	Returns:
		score: AUC score of fitted model	
	"""

    features, labels = input_fn()

    b = fit.extract(['alpha'])['alpha'].mean()
    w = fit.extract(['beta'])['beta'].mean(axis=0)

    logits = features @ w + b
    preds = expit(logits)

    score = roc_auc_score(labels, preds)
    return score

In [11]:
# !pip install stan

In [2]:
import stan

schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

posterior = stan.build(schools_code, data=schools_data)
fit = posterior.sample(num_chains=4, num_samples=1000)
eta = fit["eta"]  # array with shape (8, 4000)
df = fit.to_frame()  # pandas `DataFrame, requires pandas

RuntimeError: asyncio.run() cannot be called from a running event loop